In [1]:
# Inspiration :D
# https://gitlab.com/ryanlogsdon/pubsub-for-python/-/blob/main/subscriber.py
# https://www.youtube.com/watch?v=xOtrCmPjal8&ab_channel=D-I-Ry
import os
from google.cloud import pubsub_v1
import sys
import time
import pandas as pd
import http.client
import numpy as np
import datetime
from io import StringIO
import functools as ft

In [2]:
! python --version

Python 3.7.4


In [3]:
# Parameters section

location = "Hamburg"

## Create connection to PubSub

In [4]:
credentials_path = 'weather-based-forecasting-v2-c4bde37656a7.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_path

In [5]:
publisher_obs = pubsub_v1.PublisherClient(
    publisher_options = pubsub_v1.types.PublisherOptions(
        enable_message_ordering=True,
    )
)

publisher_forecast = pubsub_v1.PublisherClient(
    publisher_options = pubsub_v1.types.PublisherOptions(
        enable_message_ordering=True,
    )
)

topic_path_obs = 'projects/weather-based-forecasting-v2/topics/observations_topic'
topic_path_forecast = 'projects/weather-based-forecasting-v2/topics/forecasts_topic'

## Download the data

In [6]:
time.sleep(5)



conn = http.client.HTTPSConnection("visual-crossing-weather.p.rapidapi.com")

headers = {
    "X-RapidAPI-Key": "c7b48b7c4emsh8a3b688e71b9896p1be4d0jsnda2e5b27b486",
    "X-RapidAPI-Host": "visual-crossing-weather.p.rapidapi.com"
}


conn.request("GET", "/forecast?location={}&aggregateHours=1&shortColumnNames=0&unitGroup=metric&contentType=csv".format(location),
             headers=headers)

res = conn.getresponse()
data = res.read()
s = str(data, 'utf-8')
data_string = StringIO(s)
df = pd.read_csv(data_string)

In [7]:
df2 = df.copy()
df2.index = pd.to_datetime(df2["Date time"])
df2 = df2.asfreq(freq="1min")
df2 = df2.interpolate(method ='linear', limit_direction ='forward')

df2["Date time"] = df2.index

df3 = df2.iloc[:60*24*10, :]

In [8]:
df3

,Address,Date time,Latitude,Longitude,Resolved Address,Name,Wind Direction,Temperature,Wind Speed,Cloud Cover,Heat Index,Chance Precipitation (%),Precipitation,Sea Level Pressure,Snow Depth,Snow,Relative Humidity,Wind Gust,Wind Chill,Conditions
Date time,,,,,,,,,,,,,,,,,,,,
2022-12-30 21:00:00,Hamburg,2022-12-30 21:00:00,53.5537,9.99183,"Hamburg, Deutschland",Hamburg,187.100000,8.400000,20.500000,100.0,NaN,100.0,0.100000,999.000000,0.0,0.0,81.400000,47.500,5.300000,"Rain, Overcast"
2022-12-30 21:01:00,NaN,2022-12-30 21:01:00,53.5537,9.99183,NaN,NaN,187.365000,8.415000,20.495000,100.0,NaN,100.0,0.098333,999.016667,0.0,0.0,81.421667,47.470,5.318333,NaN
2022-12-30 21:02:00,NaN,2022-12-30 21:02:00,53.5537,9.99183,NaN,NaN,187.630000,8.430000,20.490000,100.0,NaN,100.0,0.096667,999.033333,0.0,0.0,81.443333,47.440,5.336667,NaN
2022-12-30 21:03:00,NaN,2022-12-30 21:03:00,53.5537,9.99183,NaN,NaN,187.895000,8.445000,20.485000,100.0,NaN,100.0,0.095000,999.050000,0.0,0.0,81.465000,47.410,5.355000,NaN
2022-12-30 21:04:00,NaN,2022-12-30 21:04:00,53.5537,9.99183,NaN,NaN,188.160000,8.460000,20.480000,100.0,NaN,100.0,0.093333,999.066667,0.0,0.0,81.486667,47.380,5.373333,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-09 20:55:00,NaN,2023-01-09 20:55:00,53.5537,9.99183,NaN,NaN,181.566667,5.008333,27.316667,100.0,NaN,57.1,0.000000,993.166667,0.1,0.0,89.275000,60.375,0.275000,NaN
2023-01-09 20:56:00,NaN,2023-01-09 20:56:00,53.5537,9.99183,NaN,NaN,181.733333,5.026667,27.293333,100.0,NaN,57.1,0.000000,993.133333,0.1,0.0,89.220000,60.320,0.300000,NaN
2023-01-09 20:57:00,NaN,2023-01-09 20:57:00,53.5537,9.99183,NaN,NaN,181.900000,5.045000,27.270000,100.0,NaN,57.1,0.000000,993.100000,0.1,0.0,89.165000,60.265,0.325000,NaN


In [9]:
now = datetime.datetime.now()
minutes = now.minute
df3_hour = df3.iloc[minutes-1:minutes+59, :]
df3_hour.head(5)

,Address,Date time,Latitude,Longitude,Resolved Address,Name,Wind Direction,Temperature,Wind Speed,Cloud Cover,Heat Index,Chance Precipitation (%),Precipitation,Sea Level Pressure,Snow Depth,Snow,Relative Humidity,Wind Gust,Wind Chill,Conditions
Date time,,,,,,,,,,,,,,,,,,,,
2022-12-30 21:30:00,NaN,2022-12-30 21:30:00,53.5537,9.99183,NaN,NaN,195.050,8.850,20.350,100.0,NaN,100.0,0.050000,999.500000,0.0,0.0,82.050000,46.60,5.850000,NaN
2022-12-30 21:31:00,NaN,2022-12-30 21:31:00,53.5537,9.99183,NaN,NaN,195.315,8.865,20.345,100.0,NaN,100.0,0.048333,999.516667,0.0,0.0,82.071667,46.57,5.868333,NaN
2022-12-30 21:32:00,NaN,2022-12-30 21:32:00,53.5537,9.99183,NaN,NaN,195.580,8.880,20.340,100.0,NaN,100.0,0.046667,999.533333,0.0,0.0,82.093333,46.54,5.886667,NaN
2022-12-30 21:33:00,NaN,2022-12-30 21:33:00,53.5537,9.99183,NaN,NaN,195.845,8.895,20.335,100.0,NaN,100.0,0.045000,999.550000,0.0,0.0,82.115000,46.51,5.905000,NaN
2022-12-30 21:34:00,NaN,2022-12-30 21:34:00,53.5537,9.99183,NaN,NaN,196.110,8.910,20.330,100.0,NaN,100.0,0.043333,999.566667,0.0,0.0,82.136667,46.48,5.923333,NaN


## Send observations in intervals to PubSub - check Weather_PubSub_check for results

In [13]:
for i in range(3):
    obs = str(df3_hour.iloc[i,:].astype(str).to_dict())
    obs = obs.replace("'", '"')
    index = df3_hour.iloc[i,:].name
    index = str(int(index.value / 10**9)).encode("utf-8")
    obs = obs.encode("utf-8")
    future = publisher_obs.publish(topic_path_obs, obs, ordering_key=index)
    print(f'published message id {future.result()}')
    time.sleep(60)

published message id 6684123182465962
published message id 6684123182609521
published message id 6684122402173674
